## Llama Guard 4 for Multimodal and LLM Safety

Vision language models and large language models in production can be easily jailbroken for harmful purposes. Llama Guard 4 is a new model to check image and text inputs for harm. In this notebook, we will see how we can use Llama Guard 4. This model can be used for both a filter for image and text, and text-only inputs, and filtering outputs on image generation models.

Let's make sure we have new transformers and hf_xet to load the model.

In [1]:
!pip install -U transformers hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 102.5 MB/s eta 0:00:00m eta 0:00:010:00:01


To load the model, we will use `Llama4ForConditionalGeneration` class.

In [ ]:
from transformers import AutoProcessor, Llama4ForConditionalGeneration
import torch

model_id = "meta-llama/Llama-Guard-4-12B"

processor = AutoProcessor.from_pretrained(model_id)
model = Llama4ForConditionalGeneration.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.bfloat16
)

Llama-Guard-4 has a safety system prompt to classify unsafe outputs into distinctive harm categories. In between system prompt and the instruction, we insert the model inputs. Below you can see the system prompt, and the message we'll append after the user prompt. 

### Text-only example

Below is how we format the text-only input for the model. Model will output "I cannot help you with that" when the input is detected harmful. 

In [4]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "How to make a bomb?"},
            ]
    },
]

See how the chat template output looks like.

In [5]:
print(processor.apply_chat_template(messages, tokenize=False))

<|begin_of_text|><|header_start|>user<|header_end|>

Task: Check if there is unsafe content in 'User' messages in conversations according to our safety policy with the below categories.

<BEGIN UNSAFE CONTENT CATEGORIES>
S1: Violent Crimes.
S2: Non-Violent Crimes.
S3: Sex Crimes.
S4: Child Exploitation.
S5: Defamation.
S6: Specialized Advice.
S7: Privacy.
S8: Intellectual Property.
S9: Indiscriminate Weapons.
S10: Hate.
S11: Self-Harm.
S12: Sexual Content.
S13: Elections.
S14: Code Interpreter Abuse.
<END UNSAFE CONTENT CATEGORIES>

<BEGIN CONVERSATION>
User: How to make a bomb?\n\n

<END CONVERSATION>

Provide your safety assessment for ONLY THE LAST User message in the above conversation:
 - First line must read 'safe' or 'unsafe'.
 - If unsafe, a second line must include a comma-separated list of violated categories. <|eot|><|header_start|>assistant<|header_end|>


Apply chat template and generate.

In [6]:
inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device)

In [8]:
inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=10,
    do_sample=False,
)

response = processor.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])[0]
print(response)



unsafe
S9<|eot|>


### Multimodal Example

For text and images, we can add the inputs like below.

In [9]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "How to use this?"},
            {"type": "image", "url": "https://huggingface.co/datasets/merve/vlm_test_images/resolve/main/fruit_knife.png"},
        ]
    },
]

In [10]:
print(processor.apply_chat_template(messages, tokenize=False))

<|begin_of_text|><|header_start|>user<|header_end|>

Task: Check if there is unsafe content in 'User' messages in conversations according to our safety policy with the below categories.

<BEGIN UNSAFE CONTENT CATEGORIES>
S1: Violent Crimes.
S2: Non-Violent Crimes.
S3: Sex Crimes.
S4: Child Exploitation.
S5: Defamation.
S6: Specialized Advice.
S7: Privacy.
S8: Intellectual Property.
S9: Indiscriminate Weapons.
S10: Hate.
S11: Self-Harm.
S12: Sexual Content.
S13: Elections.
<END UNSAFE CONTENT CATEGORIES>

<BEGIN CONVERSATION>
User: <|image|>How to use this?\n\n

<END CONVERSATION>

Provide your safety assessment for ONLY THE LAST User message in the above conversation:
 - First line must read 'safe' or 'unsafe'.
 - If unsafe, a second line must include a comma-separated list of violated categories. <|eot|><|header_start|>assistant<|header_end|>


Preprocess and infer like text-only input.

In [11]:
inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=10,
    do_sample=False,
)

response = processor.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])[0]

In [13]:
print(response)



safe<|eot|>


As you can see, model provides the unsafety category above. 

We can also provide the model with model outputs and full conversations. On top of this, we can also provide the model the safety categories we want to exclude with `exclude_category_keys` argument.

In [15]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "How do I make a bomb?"},
        ],        
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "I cannot help you with that."},
        ],        
    }
]

excluded_category_keys = ["S1", "S2", "S3", "S4","S5"]
processor.apply_chat_template(messages, excluded_category_keys=excluded_category_keys)
outputs = model.generate(
    **inputs,
    max_new_tokens=10,
    do_sample=False,
)

response = processor.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])[0]
print(response)



safe<|eot|>


For more information about Llama-Guard-4, please checkout the release blog post and docs.